In [2]:
from dataclasses import dataclass
from pathlib import Path
# entity
@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path :Path
    params_image_size : list
    params_learning_rate : float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from ChestDiseasePrediction.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from ChestDiseasePrediction.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> BaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories[config.root_dir]
        
        prepare_base_model_config = BaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes= self.params.CLASSES
        )
        return prepare_base_model_config

In [7]:
import os
from urllib import request
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self,config=BaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            input_shape= self.config.params_image_size
        )
        self.save_model(Path = self.config.base_model_path,model=self.model)

    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
            )
        self.full_model.save_model(path=self.config.updated_base_model_path,model=self.full_model)
    
    @staticmethod
    def save_model(path : Path,model:tf.keras.Model):
        model.save(path)

In [8]:
try:
    configmanager = ConfigurationManager()
    prepare_base_model_config = configmanager.get_prepare_base_model_config()
    base_model = PrepareBaseModel(prepare_base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'